In [1]:
#default_exp model

In [2]:
#export
from IPython.display import Image, display
import glob
import os
import pandas as pd
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from toolz import compose
import unet
from tensorflow.keras import losses, metrics, layers, models
from deeplearning_image_pixelwise import data, config


gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
#export
DATA_DIR = config.DATA_DIR
TRAIN_DIR = config.TRAIN_DIR
TRAIN_MASK_DIR = config.TRAIN_MASK_DIR
VAL_DIR = config.VAL_DIR
VAL_MASK_DIR = config.VAL_MASK_DIR
N_CLASSES = config.N_CLASSES
BATCH_SIZE = config.BATCH_SIZE
IMG_HEIGHT, IMG_WIDTH = config.IMG_WIDTH, config.IMG_WIDTH
float_dtype = config.float_dtype

buffer_size = 32 

In [4]:
#export
base_n_filters = 16 
BASE_DROPOUT_RATE = 0.1
ACTIVATION = 'relu'
INITIALIZER = 'he_normal'

In [5]:
tf.keras.backend.set_floatx(float_dtype)

In [6]:
%matplotlib inline
%cd ..

/home/kuba/Projects/deeplearning_image_pixelwise


In [7]:
val_dataset = data.load_dataset(VAL_DIR, VAL_MASK_DIR)
train_dataset = data.load_dataset(TRAIN_MASK_DIR, TRAIN_MASK_DIR)

In [8]:
def unet_forward_block(input_, n_filters, dropout_rate, activation, initializer):
    conv_out = layers.Conv2D(n_filters, (3, 3), activation=activation, kernel_initializer=initializer, padding='same')(input_)
    conv_out = layers.BatchNormalization()(conv_out)
    conv_out = layers.Dropout(dropout_rate) (conv_out)
    conv_out = layers.Conv2D(n_filters, (3, 3), activation=activation, kernel_initializer=initializer, padding='same') (conv_out)
    conv_out = layers.BatchNormalization()(conv_out)
    pool_out = layers.MaxPooling2D((2, 2)) (conv_out)
    return conv_out, pool_out 
    
    
def unet_skip_connect_block(current, skip_connected, n_filters, dropout_rate, activation, initializer):
    conv_current = layers.Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same') (current)
    skip_connected_concat = layers.concatenate([conv_current, skip_connected])
    skip_connected_concat = layers.Conv2D(2 * n_filters, (3, 3), activation=activation, kernel_initializer=initializer, padding='same') (skip_connected_concat)
    skip_connected_concat = layers.BatchNormalization()(skip_connected_concat)
    skip_connected_concat = layers.Dropout(dropout_rate) (skip_connected_concat)
    skip_connected_concat = layers.Conv2D(2 * n_filters, (3, 3), activation=activation, kernel_initializer=initializer, padding='same') (skip_connected_concat)
    return layers.BatchNormalization()(skip_connected_concat)

In [9]:
def build_segmentation_model(
        input_shape,
        n_classes,
        base_n_filters=base_n_filters,
        base_dropout_rate=BASE_DROPOUT_RATE,
        activation=ACTIVATION,
        initializer=INITIALIZER
    ):
    # Build U-Net segmentation_model
    inputs = layers.Input(input_shape)
    s = layers.Lambda(lambda x: x - 0.5) (inputs)
    
    c1, p1 = unet_forward_block(s, 2 * base_n_filters, base_dropout_rate, activation, initializer)
    c2, p2 = unet_forward_block(p1, 4 * base_n_filters, base_dropout_rate, activation, initializer)
    c3, p3 = unet_forward_block(p2, 8 * base_n_filters, 2 * base_dropout_rate, activation, initializer)
    c4, p4 = unet_forward_block(p3, 16 * base_n_filters, 2 * base_dropout_rate, activation, initializer)
    c5, __ = unet_forward_block(p4, 32 * base_n_filters, 3 * base_dropout_rate, activation, initializer)

    #concating starts
    u6 = unet_skip_connect_block(c5, c4, 2 * base_n_filters, base_dropout_rate, activation, initializer)
    u7 = unet_skip_connect_block(u6, c3, 4 * base_n_filters, 2 * base_dropout_rate, activation, initializer)
    u8 = unet_skip_connect_block(u7, c2, 2 * base_n_filters, base_dropout_rate, activation, initializer)
    u9 = unet_skip_connect_block(u8, c1, base_n_filters, base_dropout_rate, activation, initializer)

    out = layers.Conv2D(n_classes, (1, 1)) (u9)
    return models.Model(inputs=[inputs], outputs=[out])


def setup_segmentation_model(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        n_classes=N_CLASSES,
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer='adam',
        metrics=['accuracy']
    ):
    segmentation_model = build_segmentation_model(input_shape, n_classes)
    segmentation_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return segmentation_model

In [10]:
segmentation_model = setup_segmentation_model()
segmentation_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 32) 896         lambda[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 32) 128         conv2d[0][0]                     
______________________________________________________________________________________________

In [ ]:
batch_size = 8
epochs = 2

segmentation_model.fit(
    train_dataset.batch(batch_size).shuffle(buffer_size), 
    validation_data=val_dataset.batch(batch_size),
    epochs=epochs
)